In [2]:
!pip install -q keras

In [3]:
pip install mtcnn

Note: you may need to restart the kernel to use updated packages.


In [7]:
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN
import matplotlib.pyplot as plt
import imageio

INPUT_IMAGE = '50.png'
#OUTPUT_IMAGE = 'mingyu_alignment.png'
detector = MTCNN(steps_threshold=[0.0, 0.0, 0.0])

#img = cv2.cvtColor(cv2.imread(INPUT_IMAGE), cv2.COLOR_BGR2RGB)
detector = MTCNN()
#detections = detector.detect_faces(img)
#detections


In [ ]:
img_with_dets = cv2.imread("50.png")
min_conf = 0.9
for det in detections:
    if det['confidence'] >= min_conf:
        x, y, width, height = det['box']
        keypoints = det['keypoints']
        cv2.rectangle(img_with_dets, (x,y), (x+width,y+height), (0,155,255), 2)
        cv2.circle(img_with_dets, (keypoints['left_eye']), 2, (0,155,255), 2)
        cv2.circle(img_with_dets, (keypoints['right_eye']), 2, (0,155,255), 2)
        cv2.circle(img_with_dets, (keypoints['nose']), 2, (0,155,255), 2)
        cv2.circle(img_with_dets, (keypoints['mouth_left']), 2, (0,155,255), 2)
        cv2.circle(img_with_dets, (keypoints['mouth_right']), 2, (0,155,255), 2)
plt.figure(figsize = (10,10))
plt.imshow(img_with_dets)
plt.axis('off')

In [ ]:
def landmarks(img):
    faces = detector.detect_faces(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    face = max(faces, key=lambda x: x['confidence'])
    return face['keypoints']

def affineMatrix(lmks, scale=4):
    nose = np.array(lmks['nose'], dtype=np.float32)
    left_eye = np.array(lmks['left_eye'], dtype=np.float32)
    right_eye = np.array(lmks['right_eye'], dtype=np.float32)
    eye_width = right_eye - left_eye
    angle = np.arctan2(eye_width[1], eye_width[0])
    center = nose
    alpha = np.cos(angle)
    beta = np.sin(angle)
    w = np.sqrt(np.sum(eye_width**2)) * scale
    m = [[alpha, beta, -alpha * center[0] - beta * center[1] + w * 0.4],
        [-beta, alpha, beta * center[0] - alpha * center[1] + w * 0.4]]
    return np.array(m), (int(w), int(w))

if __name__ == '__main__':
    img = cv2.imread(INPUT_IMAGE)
    mat, size = affineMatrix(landmarks(img))
    cv2.imwrite(OUTPUT_IMAGE, cv2.warpAffine(img, mat, size))

In [ ]:
#https://github.com/Ali-Jakhar/Face-detection-using-MTCNN
#動態顯示
import cv2
video = cv2.VideoCapture("shake_it_up_2.mp4")
if (video.isOpened() == False):
    print("Error reading video file")

# We need to set resolutions.
# so, convert them from float to integer.
frame_width = int(video.get(3))
frame_height = int(video.get(4))

size = (frame_width, frame_height)

# Below VideoWriter object will create
# a frame of above defined The output
# is stored in 'filename.avi' file.
result = cv2.VideoWriter('shake_it_up.avi',cv2.VideoWriter_fourcc(*'MJPG'),29, size)
frame_num=0
while (True):
    ret, frame = video.read()
    frame_num += 1
    print(frame_num)
    if ret == True:

        location = detector.detect_faces(frame)
        if len(location) > 0:
            for face in location:
                x, y, width, height = face['box']
                x2, y2 = x + width, y + height
                cv2.rectangle(frame, (x, y), (x2, y2), (0, 0, 255), 4)
        result.write(frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Break the loop
    else:
        break


video.release()
result.release()

# Closes all the frames
cv2.destroyAllWindows()

print("The video was successfully saved")

In [8]:
def faceDetection(frame):
    detector = MTCNN()
    boxes = detector.detect_faces(frame)
    if boxes:
        box = boxes[0]['box']
        conf = boxes[0]['confidence']
        if conf > 0.9:
            x, y, w, h = box[0], box[1], box[2], box[3]
            return x, y, w, h
    else:
        return None


In [ ]:
frame_reader = imageio.get_reader('shake_it_up_2.mp4','ffmpeg', mode ='I')
for frame_num, frame in enumerate(frame_reader):
  if faceDetection(frame)!= None:
    x, y, w, h = faceDetection(frame)
    fig = plt.figure()
    plt.axis('off')
    plt.imshow(frame[y:y+h,x:x+w,:])
    plt.show()
    fig.savefig('output.png',bbox_inches='tight',pad_inches=0) #白色編框寬度=0
    break

In [11]:
import cv2

def cutVideo():
    i=0
    video = cv2.VideoCapture("FreshOffTheBoat.mp4")
    if video.isOpened():
        frame = 0
    while(True):
        ret,frame = video.read()
        #cv2.imshow('video',frame)
        c = cv2.waitKey(100)
        
        if c == 27: 
            break
        i += 1
        if i%10==0:
            if(frame.all()!=None):
                cv2.imwrite( str(i) + '.png',frame)
            '''if(faceDetection(frame)!=None):
                x, y, w, h = faceDetection(frame)
                fig = plt.figure()
                plt.axis('off')
                plt.imshow(frame[y:y+h,x:x+w,:])
                fig.savefig('output' + str(i) +'.png',bbox_inches='tight',pad_inches=0) '''


cutVideo()
cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'all'